# Imports

In [ ]:
import os
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt

In [ ]:
from Nim.Nim import Nim
from Nim.NimLogic import NimLogic

from Agents.Minimax.MinimaxAgentV1 import MinimaxAgentV1
from Agents.Minimax.MinimaxAgentV2 import MinimaxAgentV2
from Agents.Minimax.MinimaxAgentV3 import MinimaxAgentV3

Random seed for reproducibility

In [ ]:
np.random.seed(42)

# Agent Testing Function

In [ ]:
def test_agent(_misereAgent, _normalAgent, _misere, _initial_piles):
    explored_nodes = np.ndarray(EPISODES)
    moves_count = np.ndarray(EPISODES)
    mean_nodes = np.ndarray(EPISODES)

    for i in tqdm(range(EPISODES)):
        game = Nim(
            initial_piles=_initial_piles[i],
            misere=_misere[i]
        )

        agent = _misereAgent if _misere[i] else _normalAgent

        winner = game.play(
            player1=agent,
            player2=agent,
            verbose=False
        )

        assert winner == NimLogic.is_p_position(_initial_piles[i], _misere[i])

        agent.compute_mean_nodes()

        mean_nodes[i] = agent.mean_nodes
        moves_count[i] = agent.moves_count
        explored_nodes[i] = agent.nodes_explored

    weighted_mean_nodes = explored_nodes.sum() / moves_count.sum()
    unweighted_mean_nodes = mean_nodes.mean()

    avg_explored_nodes = explored_nodes.mean()
    avg_moves_count = moves_count.mean()

    values = [unweighted_mean_nodes, weighted_mean_nodes, avg_explored_nodes, avg_moves_count]

    labels = [
        "unweighted average explored nodes per move:",
        "weighted average explored nodes per move:",
        "average explored nodes per game:",
        "average moves per game:"
    ]

    label_width = max(len(lbl) for lbl in labels)

    for lbl, val in zip(labels, values):
        print(f"{lbl:<{label_width}} {val:>10.2f}")

    return explored_nodes, moves_count, mean_nodes

# Test running function

In [ ]:
def run_tests(agent_v1, agent_v2, agent_v3, normal_v1, normal_v2, normal_v3, max_depth, pile_count, max_pile, episodes, save_path_prefix):
    os.makedirs(save_path_prefix, exist_ok=True)

    print("-" * 60)
    classic_configuration = False

    if pile_count == 0 or max_pile == 0 or max_depth == 0:
        classic_configuration = True
        initial_piles = np.tile([1, 3, 5, 7], (episodes, 1))
        print(f"\tTestare fara max_depth, pe configuratia clasica")
    else:
        initial_piles = np.random.randint(1, max_pile, size=(episodes, pile_count))
        print(f"\tTestare cu max_depth={max_depth}, pile_count={pile_count}, max_pile={max_pile}")

    print("-" * 60)
    misere_modes = np.random.choice([True, False], size=episodes)

    results = {
        'v1': test_agent(agent_v1, normal_v1, misere_modes, initial_piles),
        'v2': test_agent(agent_v2, normal_v2, misere_modes, initial_piles),
        'v3': test_agent(agent_v3, normal_v3, misere_modes, initial_piles),
    }

    for version, result in results.items():
        if not classic_configuration:
            filename = f"{save_path_prefix}/minimax-{version}-{max_depth}-{pile_count}-{max_pile}.npz"
        else:
            filename = f"{save_path_prefix}/minimax-{version}-{max_depth}-classic.npz"
        np.savez(filename, result)

# Plotting results

In [59]:
def _first_existing_file(patterns):
    for p in patterns:
        if os.path.exists(p):
            return p
    return None

def plot_results(vary_parameter,
                 max_depth_values,
                 vary_values,
                 fixed_pile_count,
                 fixed_max_pile,
                 folder="savedData/Minimax",
                 save_folder="savedPlots/Minimax"):
    os.makedirs(save_folder, exist_ok=True)

    models       = ['v1', 'v2', 'v3']
    model_names  = ['Minimax V1', 'Minimax V2', 'Minimax V3']
    colors       = ['#4B0082', '#4169E1', '#9370DB']
    metric_labels = [
        'Avg Nodes/Move',
        'Avg Nodes/Game',
        'Avg Moves/Game'
    ]

    bar_width = 0.3
    x_indices = np.arange(len(vary_values))

    for max_depth in max_depth_values:
        fig, axes = plt.subplots(1, 3, figsize=(10, 4), facecolor='white', sharex=True)
        axes = axes.ravel()

        all_metrics = {m: {i: [] for i in range(3)} for m in models}

        for vary_value in vary_values:
            for model in models:
                pile_count = vary_value if vary_parameter == "pile_count" else fixed_pile_count
                max_pile   = fixed_max_pile if vary_parameter == "pile_count" else vary_value

                candidate_files = [
                    f"{folder}/minimax-{model}-{max_depth}-{pile_count}-{max_pile}.npz",
                    f"{folder}/minimax-{model}-{max_depth}-classic.npz",
                ]

                filename = _first_existing_file(candidate_files)

                if filename is None:
                    print(f"⚠️ Lipsă fișier pentru: depth={max_depth}, "
                          f"{vary_parameter}={vary_value} → {candidate_files[-1].split('/')[-1]}")
                    for i in range(3):
                        all_metrics[model][i].append(np.nan)
                    continue

                data                = np.load(filename)
                test_data           = data.get('arr_0')
                explored_nodes, moves_count, mean_nodes = test_data

                metrics_values = [
                    explored_nodes.sum() / moves_count.sum(),
                    explored_nodes.mean(),
                    moves_count.mean()
                ]
                for i in range(3):
                    all_metrics[model][i].append(metrics_values[i])

        for metric_idx, ax in enumerate(axes):
            for model_idx, model in enumerate(models):
                offset = -0.2 + 0.2 * model_idx
                ax.bar(
                    x_indices + offset,
                    all_metrics[model][metric_idx],
                    width=bar_width,
                    color=colors[model_idx],
                    alpha=1,
                    # alpha=0.6 + 0.2 * model_idx,
                    # edgecolor='black',
                    # linewidth=0.8,
                    label=model_names[model_idx]
                )
            ax.set_ylabel(metric_labels[metric_idx], fontsize=14)
            ax.set_xlabel(vary_parameter.upper(), fontsize=14)
            ax.set_xticks(x_indices)
            ax.set_xticklabels(vary_values)
            ax.grid(True, axis='y', alpha=0.3)
            if metric_idx == 0:
                ax.legend(loc='upper right')

        plt.tight_layout(rect=[0, 0, 1, 0.97])

        save_path = os.path.join(save_folder,
                                 f"variation_{vary_parameter}_depth_{max_depth}.png")
        plt.savefig(save_path, facecolor='white')
        plt.close()


In [ ]:
EPISODES = 10000

PILE_COUNT = 4
MAX_PILE = 127

for max_depth in [1, 2]:
    misere_agents = [
        MinimaxAgentV1(misere=True, max_depth=max_depth),
        MinimaxAgentV2(misere=True, max_depth=max_depth),
        MinimaxAgentV3(misere=True, max_depth=max_depth)
    ]

    normal_agents = [
        MinimaxAgentV1(misere=False, max_depth=max_depth),
        MinimaxAgentV2(misere=False, max_depth=max_depth),
        MinimaxAgentV3(misere=False, max_depth=max_depth)
    ]

    for pile_count in [2, 4, 8]:
        run_tests(*misere_agents, *normal_agents, max_depth, pile_count, MAX_PILE, EPISODES, "savedData/Minimax")

    for max_pile in [63, 255]:
        run_tests(*misere_agents, *normal_agents, max_depth, PILE_COUNT, max_pile, EPISODES, "savedData/Minimax")

In [ ]:
plot_results(vary_parameter="pile_count", max_depth_values=[1, 2], vary_values=[2, 4, 8], fixed_pile_count=4, fixed_max_pile=127)
plot_results(vary_parameter="max_pile", max_depth_values=[1, 2], vary_values=[63, 127, 255], fixed_pile_count=4, fixed_max_pile=127)

# Max Depth testing on Smaller Configuration

In [58]:
EPISODES = 10000

PILE_COUNT = 2
MAX_PILE = 3

MAX_DEPTH = 9999

misere_agents = [
    MinimaxAgentV1(misere=True, max_depth=MAX_DEPTH),
    MinimaxAgentV2(misere=True, max_depth=MAX_DEPTH),
    MinimaxAgentV3(misere=True, max_depth=MAX_DEPTH)
]

normal_agents = [
    MinimaxAgentV1(misere=False, max_depth=MAX_DEPTH),
    MinimaxAgentV2(misere=False, max_depth=MAX_DEPTH),
    MinimaxAgentV3(misere=False, max_depth=MAX_DEPTH)
]

for pile_count in [2, 3, 4]:
    run_tests(*misere_agents, *normal_agents, MAX_DEPTH, pile_count, MAX_PILE, EPISODES, "savedData/Minimax")

for max_pile in [3, 5, 7]:
    run_tests(*misere_agents, *normal_agents, MAX_DEPTH, PILE_COUNT, max_pile, EPISODES, "savedData/Minimax")

------------------------------------------------------------
	Testare cu max_depth=9999, pile_count=2, max_pile=3
------------------------------------------------------------


100%|██████████| 10000/10000 [00:00<00:00, 27311.43it/s]


unweighted average explored nodes per move:       6.90
weighted average explored nodes per move:         7.50
average explored nodes per game:                 19.59
average moves per game:                           2.61


100%|██████████| 10000/10000 [00:00<00:00, 30943.45it/s]


unweighted average explored nodes per move:       2.52
weighted average explored nodes per move:         2.69
average explored nodes per game:                  7.02
average moves per game:                           2.61


...

unweighted average explored nodes per move:     358.11
weighted average explored nodes per move:       634.05
average explored nodes per game:               3420.70
average moves per game:                           5.39


100%|██████████| 10000/10000 [00:09<00:00, 1053.58it/s]


unweighted average explored nodes per move:      27.73
weighted average explored nodes per move:        42.84
average explored nodes per game:                231.14
average moves per game:                           5.39


100%|██████████| 10000/10000 [00:01<00:00, 5329.81it/s]

unweighted average explored nodes per move:      14.81
weighted average explored nodes per move:        14.73
average explored nodes per game:                 44.79
average moves per game:                           3.04


In [57]:
plot_results(vary_parameter="pile_count", max_depth_values=[9999], vary_values=[2, 3, 4], fixed_pile_count=3, fixed_max_pile=5)
plot_results(vary_parameter="max_pile", max_depth_values=[9999], vary_values=[3, 5, 7], fixed_pile_count=3, fixed_max_pile=5)

# Max Depth Testing on Default Configuration (Fix for the sake of it)

In [ ]:
# run_tests(*misere_agents, *normal_agents, MAX_DEPTH, 0, 0, EPISODES, "savedData/Minimax")

In [56]:
# plot_results(vary_parameter="pile_count", max_depth_values=[9999], vary_values=[0], fixed_pile_count=0, fixed_max_pile=0)